In [1]:
#Instalación de las librerías (ejecutar solo la primera vez)
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install PyPDF2 langchain sentence-transformers chromadb ollama

In [2]:
#Importamos las librerías

from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb
import ollama


In [3]:
#Creamos la función que leerá el PDF
def leer_pdf(ruta_pdf):    
    reader = PdfReader(ruta_pdf)   # Abrimos el PDF
    texto = ""
    for pagina in reader.pages:     # Recorremos cada página
        texto += pagina.extract_text() + "\n"  # Extraemos texto y agregamos salto de línea
    return texto


In [4]:
#Dividimos el texto en chunks

def dividir_texto(texto, tamano=1000, solapamiento=100):
    splitter = RecursiveCharacterTextSplitter(chunk_size=tamano, chunk_overlap=solapamiento)
    chunks = splitter.split_text(texto)
    return chunks



In [5]:
#Creamos los embeddings

def crear_embeddings(chunks):
    modelo = SentenceTransformer('all-MiniLM-L6-v2')  # rápido y ligero. Este modelo es el encargado de generar los embeddings
    embeddings = modelo.encode(chunks)
    return embeddings




#Cada chunk se transforma en un vector que representa su significado.

#Estos vectores permiten comparar semánticamente texto y consultas.

In [6]:
#Guardamos los chunks de texto y los embeddings correspondientes a cada chunk en Chroma, una base vectorial

def guardar_en_chroma(chunks, embeddings, nombre_coleccion="chatbot_educativo"):
    client = chromadb.Client()
    
    # Si la colección ya existe, borrarla primero
    if nombre_coleccion in [col.name for col in client.list_collections()]:
        client.delete_collection(nombre_coleccion)
    
    collection = client.create_collection(name=nombre_coleccion)
    collection.add(
        documents=chunks,
        embeddings=embeddings,
        ids=[str(i) for i in range(len(chunks))]
    )
    return collection

#collection = guardar_en_chroma(chunks, embeddings) #la variable representa el almacenamiento de los chunks de texto y los embeddings correspondiente a cada chunk


#client.list_collections() y delete_collection() evitan errores si la colección ya existía.

#collection.add(...) guarda cada chunk con su embedding en la base vectorial.

#Esto permite consultas por similitud después.

In [7]:
#Creamos la función que buscará los chunks similares a la pregunta del usuario

def buscar_chunks_relevantes(collection, query, top_k=3):
    modelo = SentenceTransformer('all-MiniLM-L6-v2')
    query_embedding = modelo.encode([query])[0]
    resultados = collection.query(
        query_embeddings=[query_embedding],  # vector de la pregunta
        n_results=top_k
    )
    # resultados['documents'] contiene los chunks más relevantes
    return resultados['documents']


In [8]:
#Generar respuesta con Ollama


def responder_con_ollama(chunks_relevantes, query):
    # Unimos los chunks en un solo contexto
    contexto = "\n\n".join([texto for grupo in chunks_relevantes for texto in grupo]) #chunks_relevantes representa la lista de chunks
    
    # Creamos la instancia del cliente Ollama
    ollama_client = ollama.Client()
    
    # Llamamos al modelo
    respuesta = ollama_client.chat(
        model="gemma:7b-instruct",
        messages=[
            {"role": "system", "content": """Eres un asistente que responde preguntas basadas únicamente en el PDF proporcionado.
                Reglas:
                1. Responde siempre en español.
                2. Si la información no está en el PDF, responde exactamente:
                   "No puedo responder con la información que tengo."
                3. Nunca inventes información fuera del PDF.
                """
            },            
            {"role": "user", "content": f"Contexto:\n{contexto}\n\nPregunta: {query}"} #La variable query representa la pregunta del usuario
        ]
    )
    
    # Extraemos solo el contenido del mensaje del asistente
    try:
        return respuesta["message"]["content"]
    except (AttributeError, KeyError, TypeError):
        return str(respuesta)





#Tomamos los chunks relevantes y se los damos al modelo de Ollama como contexto.

#Ollama genera la respuesta en lenguaje natural.


In [16]:
def conversar(collection):
    print("Chatbot: ¡Hola! Puedo responder preguntas basándome en tu PDF. Escribí 'salir' para terminar.\n")
    pregunta = input("Tú: ")
    
    while pregunta.lower() != "salir": #Si se escribe "salir" se finaliza la conversación con el chatbot
        chunks_relevantes = buscar_chunks_relevantes(collection, pregunta) #Busca los chunks similares a la pregunta del usuario
        respuesta = responder_con_ollama(chunks_relevantes, pregunta) #Ollama recibe los chunks relevantes y produce la respuesta
        print(f"CHATBOT: {respuesta}\n") #Se imprime la respuesta
        pregunta = input("Tú: ") #Escribes la siguiente pregunta

    print("Chatbot: ¡Hasta luego!")


In [17]:
# Ruta de tu PDF
ruta_pdf = "El_Ciclo_del_Agua.pdf"

# Leer PDF
texto = leer_pdf(ruta_pdf)

# Dividir en chunks
chunks = dividir_texto(texto)

# Crear embeddings
embeddings = crear_embeddings(chunks)

# Guardar en Chroma
collection = guardar_en_chroma(chunks, embeddings)

conversar(collection)


Chatbot: ¡Hola! Puedo responder preguntas basándome en tu PDF. Escribí 'salir' para terminar.



Tú:  Qué es el ciclo del agua?


CHATBOT: El ciclo del agua es el proceso natural mediante el cual el agua se move continuamente a través de la Tierra y la atmósfera. Este ciclo es fundamental para mantener la vida en el planeta, ya que permite la renovación y distribución del agua en diferentes lugares.



Tú:  Cuáles son las etapas del ciclo del agua? Explicame cada una de ellas


CHATBOT: **Etapas del ciclo del agua:**

* **Evaporación:** El agua de los océanos, ríos, lagos y suelos se transforma en vapor debido al calor del sol y asciende a la atmósfera.


* **Condensación:** El vapor de agua se enfría al subir y se convierte en pequeñas gotas, formando nubes.


* **Precipitación:** Cuando las gotas de agua en las nubes se unen y aumentan de tamaño, caen a la superficie terrestre en forma de lluvia, nieve o granizo.


* **Infiltración y escorrentía:** Una parte del agua se filtra en el suelo y recarga los acuíferos, mientras que otra circula sobre la superficie hacia ríos y mares.



Tú:  Cuál es la tasa de precipitación?


CHATBOT: La información sobre la tasa de precipitación no está contenida en el PDF proporcionado, por lo que no puedo responder esta pregunta.



Tú:  Salir


Chatbot: ¡Hasta luego!
